---
# Lab Solution: Create a Yield Curve Hedge Method #
---

This notebook contains the complete solution to the Yield Curve Hedging lab exercise.

## Background: Interest Rate Risk and Hedging

This solution demonstrates how to calculate hedge ratios using the bumping method to measure sensitivity of a position to changes in different parts of the yield curve.

**Key Approach**: 
- Bump each reference instrument's yield individually
- Measure impact on position NPV
- Calculate offsetting positions in reference instruments

---

### Step 0: Import Pre-existing Classes

In [1]:
from instrument_classes import Bank_bill, Bond, Portfolio, CashFlows
from curve_classes_and_functions import ZeroCurve, YieldCurve
import pandas as pd
import copy

---

### Step 1: Create Reference Portfolio Instruments

We'll create a typical reference portfolio with bills and bonds at key maturities.

In [2]:
# Create bank bills for the short end of the curve
bill_3m = Bank_bill(face_value=100, maturity=0.25)
bill_3m.set_ytm(0.030)
bill_3m.set_cash_flows()
print(f"3-month bill: Maturity={bill_3m.maturity}, YTM={bill_3m.ytm:.2%}, Price=${bill_3m.price:.4f}")

bill_6m = Bank_bill(face_value=100, maturity=0.5)
bill_6m.set_ytm(0.032)
bill_6m.set_cash_flows()
print(f"6-month bill: Maturity={bill_6m.maturity}, YTM={bill_6m.ytm:.2%}, Price=${bill_6m.price:.4f}")

# Create bonds for the longer maturities
bond_1y = Bond()
bond_1y.set_face_value(100)
bond_1y.set_maturity(1)
bond_1y.set_coupon(0.035)
bond_1y.set_frequency(2)
bond_1y.set_ytm(0.035)
bond_1y.set_cash_flows()
print(f"1-year bond: Maturity={bond_1y.maturity}, YTM={bond_1y.ytm:.2%}, Price=${bond_1y.price:.4f}")

bond_2y = Bond()
bond_2y.set_face_value(100)
bond_2y.set_maturity(2)
bond_2y.set_coupon(0.040)
bond_2y.set_frequency(1)  # Changed to semi-annual for proper bootstrap
bond_2y.set_ytm(0.040)
bond_2y.set_cash_flows()
print(f"2-year bond: Maturity={bond_2y.maturity}, YTM={bond_2y.ytm:.2%}, Price=${bond_2y.price:.4f}")

bond_3y = Bond()
bond_3y.set_face_value(100)
bond_3y.set_maturity(3)
bond_3y.set_coupon(0.042)
bond_3y.set_frequency(1)  # Changed to semi-annual for proper bootstrap
bond_3y.set_ytm(0.042)
bond_3y.set_cash_flows()
print(f"3-year bond: Maturity={bond_3y.maturity}, YTM={bond_3y.ytm:.2%}, Price=${bond_3y.price:.4f}")

3-month bill: Maturity=0.25, YTM=3.00%, Price=$99.2556
6-month bill: Maturity=0.5, YTM=3.20%, Price=$98.4252
1-year bond: Maturity=1, YTM=3.50%, Price=$100.0000
2-year bond: Maturity=2, YTM=4.00%, Price=$100.0000
3-year bond: Maturity=3, YTM=4.20%, Price=$100.0000


---

### Step 2: Build the Reference Portfolio and Yield Curve

In [3]:
# Create portfolio with reference instruments
ref_portfolio = Portfolio()
ref_portfolio.add_bank_bill(bill_3m)
ref_portfolio.add_bank_bill(bill_6m)
ref_portfolio.add_bond(bond_1y)
ref_portfolio.add_bond(bond_2y)
ref_portfolio.add_bond(bond_3y)

# Build yield curve
yc = YieldCurve()
yc.set_constituent_portfolio(ref_portfolio)
yc.bootstrap()

print("\nBootstrapped Yield Curve:")
print("="*50)
maturities, discount_factors = yc.get_zero_curve()
for mat, df in zip(maturities, discount_factors):
    zero_rate = yc.get_zero_rate(mat)
    print(f"Maturity: {mat:5.2f} years | Zero Rate: {zero_rate:6.2%} | DF: {df:.6f}")


Bootstrapped Yield Curve:
Maturity:  0.00 years | Zero Rate:  0.00% | DF: 1.000000
Maturity:  0.25 years | Zero Rate:  2.99% | DF: 0.992556
Maturity:  0.50 years | Zero Rate:  3.17% | DF: 0.984252
Maturity:  1.00 years | Zero Rate:  3.47% | DF: 0.965873
Maturity:  2.00 years | Zero Rate:  3.93% | DF: 0.924390
Maturity:  3.00 years | Zero Rate:  4.13% | DF: 0.883502


---

### Step 3: Create a Position to Hedge

We'll create a simple cash flow position landing on the 3-year maturity point.

In [4]:
# Import CashFlows class
from instrument_classes import CashFlows

# Create a position with a single cash flow at the 3-year maturity
position = CashFlows()
position.add_cash_flow(3.0, 1000)  # $1000 cash flow at 3 years

# Calculate NPV using our yield curve
position_npv = yc.npv(position)

print("\nPosition to Hedge:")
print("="*50)
print(f"Cash Flow Amount: ${position.get_amounts()[0]:.2f}")
print(f"Cash Flow Maturity: {position.get_maturities()[0]} years")
print(f"NPV (using yield curve): ${position_npv:.2f}")
print(f"Discount Factor at 3Y: {yc.get_discount_factor(3.0):.6f}")


Position to Hedge:
Cash Flow Amount: $1000.00
Cash Flow Maturity: 3.0 years
NPV (using yield curve): $883.50
Discount Factor at 3Y: 0.883502


---

### Step 4: Implement the calculate_hedge() Method

Here's the complete implementation of the hedging method.

In [5]:
class YieldCurveWithHedge(YieldCurve):
    """
    Extends YieldCurve to add hedging capabilities using the bumping method.

    This class calculates hedge positions by bumping each reference instrument's
    yield and measuring the impact on the position being hedged.
    """

    def calculate_hedge(self, position, bump_size=0.0001):
        """
        Calculate hedge positions for a given position using yield curve bumping.

        The method bumps each reference instrument's yield individually, rebuilds
        the yield curve, and measures the impact on the position's NPV. The hedge
        ratios are calculated to offset these sensitivities.

        Args:
            position: An instrument object (Bond, Bank_bill, Portfolio, CashFlows, etc.)
                     with cash flows that can be valued using the yield curve
            bump_size: The yield bump size in decimal (default 0.0001 = 1 basis point)

        Returns:
            tuple: (hedge_portfolio, hedge_details)
                - hedge_portfolio: A Portfolio object containing the hedge instruments
                - hedge_details: A list of dictionaries with detailed hedge information:
                    * 'Instrument': Name of the hedge instrument
                    * 'Maturity': Maturity of the hedge instrument
                    * 'Delta Position NPV': Change in position NPV per bump
                    * 'Delta Instrument Value': Change in instrument value per bump
                    * 'Hedge Ratio': Number of units to hedge (negative = short position)
                    * 'Hedge Notional': Notional amount of the hedge

        Example:
            >>> position = CashFlows()
            >>> position.add_cash_flow(3.0, 1000)
            >>> yc = YieldCurveWithHedge()
            >>> yc.set_constituent_portfolio(ref_portfolio)
            >>> yc.bootstrap()
            >>> hedge_portfolio, hedge_details = yc.calculate_hedge(position)
        """
        # Calculate base NPV of position with current curve
        base_npv = self.npv(position)

        hedge_details = []
        hedge_portfolio = Portfolio()

        # Get reference instruments
        bank_bills = self.portfolio.get_bank_bills()
        bonds = self.portfolio.get_bonds()

        # Process each bank bill
        for i, bill in enumerate(bank_bills):
            # Create a deep copy of the bill to bump
            bumped_bill = copy.deepcopy(bill)

            # Calculate original value
            original_value = bill.price

            # IMPORTANT: Clear existing cashflows before recalculating
            # This prevents duplicate cashflows when set_cash_flows() is called
            bumped_bill.maturities = []
            bumped_bill.amounts = []

            # Bump the yield and recalculate cash flows
            bumped_bill.set_ytm(bill.ytm + bump_size)
            bumped_bill.set_cash_flows()
            bumped_value = bumped_bill.price

            # Calculate change in bill value
            delta_bill_value = bumped_value - original_value

            # Create new portfolio with bumped bill (all other instruments unchanged)
            bumped_portfolio = Portfolio()
            for j, b in enumerate(bank_bills):
                if j == i:
                    bumped_portfolio.add_bank_bill(bumped_bill)
                else:
                    bumped_portfolio.add_bank_bill(copy.deepcopy(b))
            for b in bonds:
                bumped_portfolio.add_bond(copy.deepcopy(b))

            # Rebuild yield curve with bumped instrument
            bumped_yc = YieldCurve()
            bumped_yc.set_constituent_portfolio(bumped_portfolio)
            bumped_yc.bootstrap()

            # Calculate new NPV of position
            bumped_npv = bumped_yc.npv(position)
            delta_position_npv = bumped_npv - base_npv

            # Calculate hedge ratio (negative for offsetting position)
            hedge_ratio = -delta_position_npv / delta_bill_value if delta_bill_value != 0 else 0

            # Add to hedge portfolio
            hedge_bill = copy.deepcopy(bill)
            hedge_portfolio.add_bank_bill(hedge_bill)

            # Store details
            hedge_details.append({
                'Instrument': f'Bill {bill.maturity}Y',
                'Maturity': bill.maturity,
                'Delta Position NPV': delta_position_npv,
                'Delta Instrument Value': delta_bill_value,
                'Hedge Ratio': hedge_ratio,
                'Hedge Notional': hedge_ratio * bill.face_value
            })

        # Process each bond
        for i, bond in enumerate(bonds):
            # Create a deep copy of the bond to bump
            bumped_bond = copy.deepcopy(bond)

            # Calculate original value
            original_value = bond.price

            # IMPORTANT: Clear existing cashflows before recalculating
            bumped_bond.maturities = []
            bumped_bond.amounts = []

            # Bump the yield and recalculate cash flows
            bumped_bond.set_ytm(bond.ytm + bump_size)
            bumped_bond.set_cash_flows()
            bumped_value = bumped_bond.price

            # Calculate change in bond value
            delta_bond_value = bumped_value - original_value

            # Create new portfolio with bumped bond (all other instruments unchanged)
            bumped_portfolio = Portfolio()
            for b in bank_bills:
                bumped_portfolio.add_bank_bill(copy.deepcopy(b))
            for j, b in enumerate(bonds):
                if j == i:
                    bumped_portfolio.add_bond(bumped_bond)
                else:
                    bumped_portfolio.add_bond(copy.deepcopy(b))

            # Rebuild yield curve with bumped instrument
            bumped_yc = YieldCurve()
            bumped_yc.set_constituent_portfolio(bumped_portfolio)
            bumped_yc.bootstrap()

            # Calculate new NPV of position
            bumped_npv = bumped_yc.npv(position)
            delta_position_npv = bumped_npv - base_npv

            # Calculate hedge ratio (negative for offsetting position)
            hedge_ratio = -delta_position_npv / delta_bond_value if delta_bond_value != 0 else 0

            # Add to hedge portfolio
            hedge_bond = copy.deepcopy(bond)
            hedge_portfolio.add_bond(hedge_bond)

            # Store details
            hedge_details.append({
                'Instrument': f'Bond {bond.maturity}Y',
                'Maturity': bond.maturity,
                'Delta Position NPV': delta_position_npv,
                'Delta Instrument Value': delta_bond_value,
                'Hedge Ratio': hedge_ratio,
                'Hedge Notional': hedge_ratio * bond.face_value
            })

        return hedge_portfolio, hedge_details

print("YieldCurveWithHedge class successfully defined!")

YieldCurveWithHedge class successfully defined!


---

### Step 5: Test the Hedge Method

Now let's calculate the hedge for our position.

In [6]:
# Create hedge-enabled yield curve
yc_hedge = YieldCurveWithHedge()
yc_hedge.set_constituent_portfolio(ref_portfolio)
yc_hedge.bootstrap()

# Calculate hedge
print("="*70)
print("CALCULATING HEDGE FOR POSITION")
print("="*70)
hedge_portfolio, hedge_details = yc_hedge.calculate_hedge(position, bump_size=0.0001)

# Display results in a DataFrame
print("\n\nHedge Analysis Results:")
print("="*70)
hedge_df = pd.DataFrame(hedge_details)
print(hedge_df.to_string(index=False))

# Summary statistics
print("\n\nHedge Summary:")
print("="*70)
print(f"Total Hedge Instruments: {len(hedge_details)}")
total_hedge_notional = sum([h['Hedge Notional'] for h in hedge_details])
print(f"Total Hedge Notional: ${total_hedge_notional:.2f}")
print(f"Largest Sensitivity: {max([abs(h['Delta Position NPV']) for h in hedge_details]):.4f}")

# Identify key hedge instruments
sorted_by_sensitivity = sorted(hedge_details, key=lambda x: abs(x['Delta Position NPV']), reverse=True)
print(f"\nMost Sensitive to: {sorted_by_sensitivity[0]['Instrument']} (Delta NPV: ${sorted_by_sensitivity[0]['Delta Position NPV']:.4f})")

CALCULATING HEDGE FOR POSITION


Hedge Analysis Results:
Instrument  Maturity  Delta Position NPV  Delta Instrument Value  Hedge Ratio  Hedge Notional
Bill 0.25Y      0.25            0.000000               -0.002463     0.000000        0.000000
 Bill 0.5Y      0.50           -0.000032               -0.004844    -0.006666       -0.666579
   Bond 1Y      1.00            0.003711               -0.009743     0.380902       38.090249
   Bond 2Y      2.00            0.007309               -0.018858     0.387568       38.756829
   Bond 3Y      3.00           -0.265266               -0.027641    -9.596929     -959.692898


Hedge Summary:
Total Hedge Instruments: 5
Total Hedge Notional: $-883.51
Largest Sensitivity: 0.2653

Most Sensitive to: Bond 3Y (Delta NPV: $-0.2653)


---

### Step 6: Verify the Hedge Effectiveness

Let's verify that our hedge works by simulating a parallel shift in yields.

In [7]:
def verify_hedge(position, hedge_details, ref_portfolio, yield_shift=0.0010):
    """
    Verify hedge effectiveness by simulating a yield curve shift.
    
    Args:
        position: The position being hedged
        hedge_details: List of hedge information from calculate_hedge
        ref_portfolio: Original reference portfolio
        yield_shift: Size of parallel yield shift (default 10 basis points)
    """
    print("\n\nHEDGE VERIFICATION")
    print("="*70)
    print(f"Simulating a {yield_shift*10000:.0f} basis point parallel shift in yields...\n")
    
    # Calculate original position NPV
    yc_original = YieldCurve()
    yc_original.set_constituent_portfolio(ref_portfolio)
    yc_original.bootstrap()
    original_npv = yc_original.npv(position)
    
    # Create shifted portfolio
    shifted_portfolio = Portfolio()
    
    # Shift all bills
    for bill in ref_portfolio.get_bank_bills():
        shifted_bill = copy.deepcopy(bill)
        # Clear cashflows before recalculating
        shifted_bill.maturities = []
        shifted_bill.amounts = []
        shifted_bill.set_ytm(bill.ytm + yield_shift)
        shifted_bill.set_cash_flows()
        shifted_portfolio.add_bank_bill(shifted_bill)
    
    # Shift all bonds
    for bond in ref_portfolio.get_bonds():
        shifted_bond = copy.deepcopy(bond)
        # Clear cashflows before recalculating
        shifted_bond.maturities = []
        shifted_bond.amounts = []
        shifted_bond.set_ytm(bond.ytm + yield_shift)
        shifted_bond.set_cash_flows()
        shifted_portfolio.add_bond(shifted_bond)
    
    # Build shifted yield curve
    yc_shifted = YieldCurve()
    yc_shifted.set_constituent_portfolio(shifted_portfolio)
    yc_shifted.bootstrap()
    
    # Calculate shifted position NPV
    shifted_npv = yc_shifted.npv(position)
    position_pnl = shifted_npv - original_npv
    
    print(f"Position P&L: ${position_pnl:.2f}")
    
    # Calculate hedge P&L
    hedge_pnl = 0
    print("\nHedge Instrument P&L:")
    
    # P&L from bills
    bills = ref_portfolio.get_bank_bills()
    shifted_bills = shifted_portfolio.get_bank_bills()
    for i, (bill, shifted_bill) in enumerate(zip(bills, shifted_bills)):
        hedge_ratio = hedge_details[i]['Hedge Ratio']
        pnl = hedge_ratio * (shifted_bill.price - bill.price)
        hedge_pnl += pnl
        print(f"  {hedge_details[i]['Instrument']}: ${pnl:8.2f} (ratio: {hedge_ratio:6.2f})")
    
    # P&L from bonds
    bonds = ref_portfolio.get_bonds()
    shifted_bonds = shifted_portfolio.get_bonds()
    for i, (bond, shifted_bond) in enumerate(zip(bonds, shifted_bonds)):
        hedge_ratio = hedge_details[len(bills) + i]['Hedge Ratio']
        pnl = hedge_ratio * (shifted_bond.price - bond.price)
        hedge_pnl += pnl
        print(f"  {hedge_details[len(bills) + i]['Instrument']}: ${pnl:8.2f} (ratio: {hedge_ratio:6.2f})")
    
    print(f"\nTotal Hedge P&L: ${hedge_pnl:.2f}")
    
    # Calculate net P&L
    net_pnl = position_pnl + hedge_pnl
    hedge_effectiveness = (1 - abs(net_pnl / position_pnl)) * 100 if position_pnl != 0 else 100
    
    print("\n" + "="*70)
    print("RESULTS:")
    print("="*70)
    print(f"Position P&L:     ${position_pnl:10.2f}")
    print(f"Hedge P&L:        ${hedge_pnl:10.2f}")
    print(f"Net P&L:          ${net_pnl:10.2f}")
    print(f"Hedge Effectiveness: {hedge_effectiveness:6.2f}%")
    print("="*70)
    
    if abs(net_pnl) < abs(position_pnl) * 0.1:  # Within 10%
        print("\nHEDGE IS EFFECTIVE! Net P&L is close to zero.")
    else:
        print("\nHedge effectiveness could be improved.")
        print("This may be due to non-parallel shifts or convexity effects.")

# Run verification
verify_hedge(position, hedge_details, ref_portfolio, yield_shift=0.0010)



HEDGE VERIFICATION
Simulating a 10 basis point parallel shift in yields...

Position P&L: $-2.54

Hedge Instrument P&L:
  Bill 0.25Y: $   -0.00 (ratio:   0.00)
  Bill 0.5Y: $    0.00 (ratio:  -0.01)
  Bond 1Y: $   -0.04 (ratio:   0.38)
  Bond 2Y: $   -0.07 (ratio:   0.39)
  Bond 3Y: $    2.65 (ratio:  -9.60)

Total Hedge P&L: $2.54

RESULTS:
Position P&L:     $     -2.54
Hedge P&L:        $      2.54
Net P&L:          $      0.00
Hedge Effectiveness: 100.00%

HEDGE IS EFFECTIVE! Net P&L is close to zero.


---

## Analysis and Insights

### What We've Learned

1. **Bumping Method**: We successfully implemented a yield curve bumping approach to measure sensitivities

2. **Key Rate Sensitivity**: Different instruments have different impacts because they affect different parts of the curve

3. **Hedge Ratios**: The calculated ratios tell us exactly how much of each reference instrument to hold

4. **Hedge Effectiveness**: Our verification shows the hedge works well for parallel shifts

### Why The Hedge Works

The hedge is effective because:
- Each reference instrument maps to a specific part of the yield curve
- By bumping individually, we capture the position's sensitivity to each maturity point
- The hedge ratios create offsetting exposures at each key rate

### Limitations

1. **Parallel Shifts Only**: This hedge works best for parallel yield curve shifts
2. **Small Moves**: The linear approximation (delta) is accurate only for small yield changes
3. **Convexity**: Large moves or non-parallel shifts would require convexity adjustments
4. **Time Decay**: Hedge ratios change over time and need rebalancing

### Real-World Applications

This technique is used by:
- Fixed income portfolio managers
- Bond trading desks
- Treasury departments
- Risk management teams

### OOP Benefits Demonstrated

- **Inheritance**: Extended YieldCurve without modifying the base class
- **Encapsulation**: Complex hedging logic is neatly contained in one method
- **Polymorphism**: Works with any instrument object (Bond, Portfolio, etc.)
- **Reusability**: The method can be applied to any position and reference portfolio

This lab demonstrates how OOP principles make complex financial calculations manageable and maintainable!